# Recursive Data Structures

In this lecture we'll return to our study of the more theoretical aspects of coding by examining the following data structures:

* binary trees
* binary search trees
* red black trees
* avl trees
* heap
* Graphs
    * depth first search
    * breadth first search
    * dijkstra algorithm


In [2]:
class Node:
    def __init__(self, data, left, right):
        self.data = data
        self.left = left
        self.right = right
    
    def __str__(self):
        return repr(self.data)
    
    
class BinaryTree:
    def __init__(self):
        self.head = None
    
    def append(self, data):
        if self.head is None:
            self.head = Node(data, None, None)
        else:
            cur = self.head
            self._append(cur, data)
    
    def _append(self, cur, data):
        if cur.left is None:
            cur.left = Node(data, None, None)
        elif cur.right:
            self._append(cur.left, data)
        else:
            cur.right = Node(data, None, None)
            
    def print_tree(self):
        if self.head is None:
            print(None)
        else:
            cur = self.head
            self._print_tree(cur)
    
    def _print_tree(self, cur):
        if cur.left:
            self._print_tree(cur.left)
        print(cur)
        if cur.right:
            self._print_tree(cur.right)
            

tree = BinaryTree()
for elem in range(10):
    tree.append(elem)

tree.print_tree()

9
7
5
8
3
6
1
4
0
2


So this is cool, now we have a recursive data structure!  But it looks like it doesn't store the data sequentially.  But no worries, we can fix that with a BinarySearchTree - this explicitly accounts for order during storage.

In [1]:
class Node:
    def __init__(self, data, left, right):
        self.data = data
        self.left = left
        self.right = right
    
    def __str__(self):
        return repr(self.data)
    
    
class BinarySearchTree:
    def __init__(self):
        self.head = None
        self.depth = 0
    
    def append(self, data):
        if self.head is None:
            self.head = Node(data, None, None)
        else:
            cur = self.head
            self._append(cur, data)
    
    def _append(self, cur, data):
        if data <= cur.data:
            if cur.left:
                self._append(cur.left, data)
            else:
                if cur.right is None:
                    self.depth += 1
                cur.left = Node(data, None, None)
        else:
            if cur.right:
                self._append(cur.right, data)
            else:
                if cur.left is None:
                    self.depth += 1
                cur.right = Node(data, None, None)
                
    def print_tree(self):
        if self.head is None:
            print(None)
        else:
            cur = self.head
            self._print_tree(cur)
    
    def _print_tree(self, cur):
        if cur.left:
            self._print_tree(cur.left)
        print(cur)
        if cur.right:
            self._print_tree(cur.right)
    
            

tree = BinarySearchTree()
for elem in range(10):
    tree.append(elem)

tree.print_tree()
tree.depth

0
1
2
3
4
5
6
7
8
9


9

In [2]:
tree = BinarySearchTree()
for elem in range(10, 1, -1):
    tree.append(elem)
    
tree.print_tree()
tree.depth

2
3
4
5
6
7
8
9
10


8

So in both cases, we get the right order, but our tree ends up being nothing more than a linked list, that's why the depth is equal to the number of elements.  This is very not good!  It means we don't take advantage of the O(log n) access time of a balanced tree.  So we'll need some more algorithms to make our binary search tree balanced.

## Enter Balanced Binary Search Trees

In this section we will cover:

* red-black trees
* AVL trees

Properties of a red-black tree:
* Every node is either red or black.
* Every leaf (NULL) is black.
* If a node is red, then both its children are black.
* Every simple path from a node to a descendant leaf contains the same number of black nodes.


In [2]:
# code comes from here: http://scottlobdell.me/2016/02/purely-functional-red-black-trees-python/
import uuid


class Color(object):
    RED = 0
    BLACK = 1


class RedBlackTree(object):

    def __init__(self, left, value, right, color=Color.RED):
        self._color = color
        self._left = left
        self._right = right
        self._value = value
        self._count = 1 + len(left) + len(right)
        self._node_uuid = uuid.uuid4()
        self.depth = max(len(left), len(right))

    def __len__(self):
        return self._count

    @property
    def uuid(self):
        return self._node_uuid

    @property
    def color(self):
        return self._color

    @property
    def value(self):
        return self._value

    @property
    def right(self):
        return self._right

    @property
    def left(self):
        return self._left

    def blacken(self):
        if self.is_red():
            return RedBlackTree(
                self.left,
                self.value,
                self.right,
                color=Color.BLACK,
            )
        return self

    def is_empty(self):
        return False

    def is_black(self):
        return self._color == Color.BLACK

    def is_red(self):
        return self._color == Color.RED

    def rotate_left(self):
        return RedBlackTree(
            RedBlackTree(
                self.left,
                self.value,
                EmptyRedBlackTree().update(self.right.left),
                color=self.color,
            ),
            self.right.value,
            self.right.right,
            color=self.right.color,
        )

    def rotate_right(self):
        return RedBlackTree(
            self.left.left,
            self.left.value,
            RedBlackTree(
                EmptyRedBlackTree().update(self.left.right),
                self.value,
                self.right,
                color=self.color,
            ),
            color=self.left.color,
        )

    def recolored(self):
        return RedBlackTree(
            self.left.blacken(),
            self.value,
            self.right.blacken(),
            color=Color.RED,
        )

    def balance(self):
        if self.is_red():
            return self

        if self.left.is_red():
            if self.right.is_red():
                return self.recolored()
            if self.left.left.is_red():
                return self.rotate_right().recolored()
            if self.left.right.is_red():
                return RedBlackTree(
                    self.left.rotate_left(),
                    self.value,
                    self.right,
                    color=self.color,
                ).rotate_right().recolored()
            return self

        if self.right.is_red():
            if self.right.right.is_red():
                return self.rotate_left().recolored()
            if self.right.left.is_red():
                return RedBlackTree(
                    self.left,
                    self.value,
                    self.right.rotate_right(),
                    color=self.color,
                ).rotate_left().recolored()
        return self

    def update(self, node):
        if node.is_empty():
            return self
        if node.value < self.value:
            return RedBlackTree(
                self.left.update(node).balance(),
                self.value,
                self.right,
                color=self.color,
            ).balance()
        return RedBlackTree(
            self.left,
            self.value,
            self.right.update(node).balance(),
            color=self.color,
        ).balance()

    def insert(self, value):
        return self.update(
            RedBlackTree(
                EmptyRedBlackTree(),
                value,
                EmptyRedBlackTree(),
                color=Color.RED,
            )
        ).blacken()

    def is_member(self, value):
        if value < self._value:
            return self.left.is_member(value)
        if value > self._value:
            return self.right.is_member(value)
        return True


class EmptyRedBlackTree(RedBlackTree):

    def __init__(self):
        self._color = Color.BLACK

    def is_empty(self):
        return True

    def insert(self, value):
        return RedBlackTree(
            EmptyRedBlackTree(),
            value,
            EmptyRedBlackTree(),
            color=Color.RED,
        )

    def update(self, node):
        return node

    def is_member(self, value):
        return False

    @property
    def left(self):
        return EmptyRedBlackTree()

    @property
    def right(self):
        return EmptyRedBlackTree()

    def __len__(self):
        return 0

In [3]:
tree = EmptyRedBlackTree().insert(0)
for elem in range(1, 10):
    tree = tree.insert(elem)

tree.depth

6

Red-Black Tree Visualization:

https://www.cs.usfca.edu/~galles/visualization/RedBlack.html

In [5]:
# Python code to insert a node in AVL tree
 
# Generic tree node class
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        self.height = 1
 
# AVL tree class which supports the 
# Insert operation
class AVL_Tree:
 
    def __init__(self):
        self.root = None
        
    # Recursive function to insert key in 
    # subtree rooted with node and returns
    # new root of subtree.
    def insert(self, root, key):
     
        # Step 1 - Perform normal BST
        if not root:
            return TreeNode(key)
        elif key < root.val:
            root.left = self.insert(root.left, key)
        else:
            root.right = self.insert(root.right, key)
 
        # Step 2 - Update the height of the 
        # ancestor node
        root.height = 1 + max(self.getHeight(root.left),
                           self.getHeight(root.right))
 
        # Step 3 - Get the balance factor
        balance = self.getBalance(root)
 
        # Step 4 - If the node is unbalanced, 
        # then try out the 4 cases
        # Case 1 - Left Left
        if balance > 1 and key < root.left.val:
            return self.rightRotate(root)
 
        # Case 2 - Right Right
        if balance < -1 and key > root.right.val:
            return self.leftRotate(root)
 
        # Case 3 - Left Right
        if balance > 1 and key > root.left.val:
            root.left = self.leftRotate(root.left)
            return self.rightRotate(root)
 
        # Case 4 - Right Left
        if balance < -1 and key < root.right.val:
            root.right = self.rightRotate(root.right)
            return self.leftRotate(root)
 
        return root
 
    def leftRotate(self, z):
 
        y = z.right
        T2 = y.left
 
        # Perform rotation
        y.left = z
        z.right = T2
 
        # Update heights
        z.height = 1 + max(self.getHeight(z.left),
                         self.getHeight(z.right))
        y.height = 1 + max(self.getHeight(y.left),
                         self.getHeight(y.right))
 
        # Return the new root
        return y
 
    def rightRotate(self, z):
 
        y = z.left
        T3 = y.right
 
        # Perform rotation
        y.right = z
        z.left = T3
 
        # Update heights
        z.height = 1 + max(self.getHeight(z.left),
                        self.getHeight(z.right))
        y.height = 1 + max(self.getHeight(y.left),
                        self.getHeight(y.right))
 
        # Return the new root
        return y
 
    def getHeight(self, root):
        if not root:
            return 0
 
        return root.height
 
    def getBalance(self, root):
        if not root:
            return 0
 
        return self.getHeight(root.left) - self.getHeight(root.right)
 
    def preOrder(self, root):
 
        if not root:
            return
 
        print("{0} ".format(root.val), end="")
        self.preOrder(root.left)
        self.preOrder(root.right)
 
    def getDepth(self, root, depth):
        if root.left:
            return self.getDepth(root.left, 1+depth)
        if root.right:
            return self.getDepth(root.right, 1+depth)
        else:
            return depth
        
# Driver program to test above function
myTree = AVL_Tree()
root = None
 
root = myTree.insert(root, 10)
root = myTree.insert(root, 20)
root = myTree.insert(root, 30)
root = myTree.insert(root, 40)
root = myTree.insert(root, 50)
root = myTree.insert(root, 25)


"""The constructed AVL Tree would be
            30
           /  \
         20   40
        /  \     \
       10  25    50"""
 
# Preorder Traversal
print("Preorder traversal of the",
      "constructed AVL tree is")
myTree.preOrder(root)
print ()
myTree.getDepth(root, 1)

Preorder traversal of the constructed AVL tree is
30 20 10 25 40 50 


3

AVL tree visualization:

https://www.cs.usfca.edu/~galles/visualization/AVLtree.html

In [7]:
class BinHeap:
    def __init__(self):
        self.heapList = [0]
        self.currentSize = 0


    def percUp(self, i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i // 2]:
                tmp = self.heapList[i // 2]
                self.heapList[i // 2] = self.heapList[i]
                self.heapList[i] = tmp
            i = i // 2

    def insert(self, k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def percDown(self, i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc

    def minChild(self, i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i*2] < self.heapList[i*2+1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMin(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)
        return retval

    def buildHeap(self, alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1

dicter = {
    2: "cookies",
    3: "ice cream",
    5: "sodas",
    6: "cakes",
    9: "sugar"
}
bh = BinHeap()
bh.buildHeap([9,5,6,2,3])

print(dicter[bh.delMin()])
print(dicter[bh.delMin()])
print(dicter[bh.delMin()])
print(dicter[bh.delMin()])
print(dicter[bh.delMin()])

cookies
ice cream
sodas
cakes
sugar


Heap Visualization:

https://www.cs.usfca.edu/~galles/visualization/Heap.html

In [12]:
class Node:
    def __init__(self, id, data, edges):
        self.id = id
        self.data = data
        self.edges = edges
        
    def __str__(self):
        return repr(self.data)
    
class Graph:
    def __init__(self):
        self.nodes = []
    
    def add_node(self, data):
        new_id = len(self.nodes)
        self.nodes.append(Node(new_id, data, None))
        
    def add_edge(self, id_one, id_two):
        if self.nodes[id_one].edges is None:
            self.nodes[id_one].edges = [id_two]
        else:    
            self.nodes[id_one].edges.append(id_two)
        if self.nodes[id_two].edges is None:
            self.nodes[id_two].edges = [id_one]
        else:
            self.nodes[id_two].edges.append(id_one)
        

g = Graph()
g.add_node(4)
g.add_node(10)
g.add_node(12)
g.add_edge(0, 1)
g.add_edge(2, 0)
g.add_edge(1, 2)

for elem in g.nodes:
    print(elem.edges)

[1, 2]
[0, 2]
[0, 1]


In [26]:
class Stack:
    def __init__(self):
        self._list = list()
        
    def push(self, data):
        self._list.append(data)
    
    def pop(self):
        return self._list.pop()
    
    def empty(self):
        return self._list == []

stack = Stack()
stack.push(10)
stack.push(2)
print(stack.pop())
print(stack.pop())

2
10


In [27]:
from collections import deque

class Queue:
    def __init__(self):
        self._queue = deque()
        
    def push(self, data):
        self._queue.appendleft(data)
        
    def pop(self):
        return self._queue.pop()
    
    def empty(self):
        return len(self._queue) == 0
    
queue = Queue()
queue.push(10)
queue.push(2)
print(queue.empty())
print(queue.pop())
print(queue.pop())


False
10
2


In [37]:
def dfs(node, graph):
    visited = []
    stack = Stack()
    stack.push(node)
    while not stack.empty():
        cur = stack.pop()
        print(cur)
        if cur.id not in visited:
            visited.append(cur.id)
            for edge in cur.edges:
                if edge not in visited:
                    stack.push(graph.nodes[edge])
        

g = Graph()
for i in range(30):
    g.add_node(i)

for i in range(29):
    g.add_edge(i, i+1)
    
for i in range(10):
    g.add_edge(i, i+3)
#g.add_edge(0, 1)
#g.add_edge(0, 2)
#g.add_edge(0, 3)
#g.add_edge(0, 4)

#g.add_edge(1, 5)
#g.add_edge(1, 6)
#g.add_edge(1, 7)

dfs(g.nodes[0], g)

0
3
6
9
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
11
8
5
2
1
4
7
10
4
7
10
10
8
7
5
4
2
1


In [38]:
def bfs(node, graph):
    visited = []
    queue = Queue()
    queue.push(node)
    while not queue.empty():
        cur = queue.pop()
        print(cur)
        if cur.id not in visited:
            visited.append(cur.id)
            for edge in cur.edges:
                if edge not in visited:
                    queue.push(graph.nodes[edge])
        

g = Graph()
for i in range(30):
    g.add_node(i)

for i in range(29):
    g.add_edge(i, i+1)
    
for i in range(10):
    g.add_edge(i, i+3)
#g.add_edge(0, 1)
#g.add_edge(0, 2)
#g.add_edge(0, 3)
#g.add_edge(0, 4)

#g.add_edge(1, 5)
#g.add_edge(1, 6)
#g.add_edge(1, 7)

bfs(g.nodes[0], g)

0
1
3
2
4
2
4
6
5
5
7
5
7
9
8
8
10
8
10
12
11
11
11
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
